# Autoencoder UNIBO Powertools Dataset

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys
import random

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Model


IS_COLAB = False
IS_TRAINING = True
RESULT_NAME = ""

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/battery-state-estimation/battery-state-estimation/"
else:
    data_path = "../../"

sys.path.append(data_path)
from data_processing.unibo_powertools_data import UniboPowertoolsData, CycleCols
from data_processing.model_data_handler import ModelDataHandler
from data_processing.prepare_rul_data import RulHandler

2023-03-10 11:54:35.052306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 11:54:35.192692: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 11:54:35.192708: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-10 11:54:36.017208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

### Config logging

In [2]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [3]:
dataset = UniboPowertoolsData(
    test_types=[],
    chunk_size=1000000,
    lines=[37, 40],
    charge_line=37,
    discharge_line=40,
    base_path=data_path
)

2023/03/10 11:54:44 [DEBUG]: Start loading data with lines: [37, 40], types: [] and chunksize: 1000000...
2023/03/10 11:55:11 [DEBUG]: Finish loading data.
2023/03/10 11:55:11 [INFO]: Loaded raw dataset A data with cycle row count: 15384908 and capacity row count: 39585
2023/03/10 11:55:11 [DEBUG]: Start cleaning cycle raw data...
2023/03/10 11:55:19 [DEBUG]: Finish cleaning cycle raw data.
2023/03/10 11:55:19 [INFO]: Removed 11 rows of abnormal cycle raw data.
2023/03/10 11:55:19 [DEBUG]: Start cleaning capacity raw data...
2023/03/10 11:55:19 [DEBUG]: Finish cleaning capacity raw data.
2023/03/10 11:55:19 [INFO]: Removed 1 rows of abnormal capacity raw data.
2023/03/10 11:55:19 [DEBUG]: Start assigning charging raw data...
2023/03/10 11:55:19 [DEBUG]: Finish assigning charging raw data.
2023/03/10 11:55:19 [INFO]: [Charging] cycle raw count: 12160812, capacity raw count: 19800
2023/03/10 11:55:19 [DEBUG]: Start assigning discharging raw data...
2023/03/10 11:55:20 [DEBUG]: Finish ass

In [4]:
train_names = [
    '000-DM-3.0-4019-S',#minimum capacity 1.48
    '001-DM-3.0-4019-S',#minimum capacity 1.81
    '002-DM-3.0-4019-S',#minimum capacity 2.06

    '009-DM-3.0-4019-H',#minimum capacity 1.41
    '010-DM-3.0-4019-H',#minimum capacity 1.44

    '014-DM-3.0-4019-P',#minimum capacity 1.7
    '015-DM-3.0-4019-P',#minimum capacity 1.76
    '016-DM-3.0-4019-P',#minimum capacity 1.56
    '017-DM-3.0-4019-P',#minimum capacity 1.29
    #'047-DM-3.0-4019-P',#new 1.98
    #'049-DM-3.0-4019-P',#new 2.19



    '007-EE-2.85-0820-S',#2.5
    '008-EE-2.85-0820-S',#2.49
    '042-EE-2.85-0820-S',#2.51

    '043-EE-2.85-0820-H',#2.31


    '040-DM-4.00-2320-S',#minimum capacity 3.75, cycles 188


    '018-DP-2.00-1320-S',#minimum capacity 1.82
    #'019-DP-2.00-1320-S',#minimum capacity 1.61
    '036-DP-2.00-1720-S',#minimum capacity 1.91
    '037-DP-2.00-1720-S',#minimum capacity 1.84
    '038-DP-2.00-2420-S',#minimum capacity 1.854 (to 0)
    '050-DP-2.00-4020-S',#new 1.81
    '051-DP-2.00-4020-S',#new 1.866    
    
]

test_names = [
    '003-DM-3.0-4019-S',#minimum capacity 1.84

    '011-DM-3.0-4019-H',#minimum capacity 1.36

    '013-DM-3.0-4019-P',#minimum capacity 1.6



    '006-EE-2.85-0820-S',# 2.621
    
    '044-EE-2.85-0820-H',# 2.43



    '039-DP-2.00-2420-S',#minimum capacity 1.93



    '041-DM-4.00-2320-S',#minimum capacity 3.76, cycles 190
]

In [5]:
dataset.prepare_data(train_names, test_names)
dataset_handler = ModelDataHandler(dataset, [
    CycleCols.VOLTAGE,
    CycleCols.CURRENT,
    CycleCols.TEMPERATURE
])

rul_handler = RulHandler()

2023/03/10 11:58:02 [DEBUG]: Start preparing data for training: ['000-DM-3.0-4019-S', '001-DM-3.0-4019-S', '002-DM-3.0-4019-S', '009-DM-3.0-4019-H', '010-DM-3.0-4019-H', '014-DM-3.0-4019-P', '015-DM-3.0-4019-P', '016-DM-3.0-4019-P', '017-DM-3.0-4019-P', '007-EE-2.85-0820-S', '008-EE-2.85-0820-S', '042-EE-2.85-0820-S', '043-EE-2.85-0820-H', '040-DM-4.00-2320-S', '018-DP-2.00-1320-S', '036-DP-2.00-1720-S', '037-DP-2.00-1720-S', '038-DP-2.00-2420-S', '050-DP-2.00-4020-S', '051-DP-2.00-4020-S'] and testing: ['003-DM-3.0-4019-S', '011-DM-3.0-4019-H', '013-DM-3.0-4019-P', '006-EE-2.85-0820-S', '044-EE-2.85-0820-H', '039-DP-2.00-2420-S', '041-DM-4.00-2320-S']...
2023/03/10 11:58:18 [DEBUG]: Finish getting training and testing charge data.
2023/03/10 11:58:27 [DEBUG]: Finish getting training and testing discharge data.
2023/03/10 11:58:27 [DEBUG]: Finish cleaning training and testing charge data.
2023/03/10 11:58:27 [DEBUG]: Finish cleaning training and testing discharge data.
2023/03/10 11:58

## Data preparation

In [6]:
(train_x, train_y_soh, test_x, test_y_soh,
  train_battery_range, test_battery_range,
  time_train, time_test, current_train, current_test) = dataset_handler.get_discharge_whole_cycle_future(train_names, test_names)

train_x = train_x[:,:284,:]
test_x = test_x[:,:284,:]
print("cut train shape {}".format(train_x.shape))
print("cut test shape {}".format(test_x.shape))


x_norm = rul_handler.Normalization()
train_x, test_x = x_norm.fit_and_normalize(train_x, test_x)

../../data_processing/model_data_handler.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  list(map(lambda data: data[:, x_indices].astype('float32'), cyc))
2023/03/10 11:58:50 [INFO]: Train x: (11325, 287, 3), train y soh: (11325, 1) | Test x: (3507, 287, 3), test y soh: (3507, 1) | 
                            battery n cycle train: (20,), battery n cycle test: (7,), 
                            time train: (11325, 287, 1), time test: (3507, 287, 1) |
                            raw current train: (11325, 287), raw current test: (3507, 287) |
                            


cut train shape (11325, 284, 3)
cut test shape (3507, 284, 3)


# Model training

In [7]:
if IS_TRAINING:
    EXPERIMENT = "autoencoder_unibo_powertools"

    experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
    print(experiment_name)

# Model definition

opt = tf.keras.optimizers.Adam(learning_rate=0.0002)
LATENT_DIM = 10

class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(train_x.shape[1], train_x.shape[2])),
            #layers.MaxPooling1D(5, padding='same'),
            layers.Conv1D(filters=16, kernel_size=5, strides=2, activation='relu', padding='same'),
            layers.Conv1D(filters=8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Flatten(),
            layers.Dense(self.latent_dim, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            layers.Input(shape=(self.latent_dim)),
            layers.Dense(568, activation='relu'),
            layers.Reshape((71, 8)),
            layers.Conv1DTranspose(filters=8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv1DTranspose(filters=16, kernel_size=5, strides=2, activation='relu', padding='same'),
            layers.Conv1D(3, kernel_size=3, activation='relu', padding='same'),
            #layers.UpSampling1D(5),
        ])


    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(LATENT_DIM)
autoencoder.compile(optimizer=opt, loss='mse', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
autoencoder.encoder.summary()
autoencoder.decoder.summary()

2023-03-10-11-59-00_autoencoder_unibo_powertools


2023-03-10 11:59:00.714480: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-10 11:59:00.714747: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-10 11:59:00.714777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nocworkp1): /proc/driver/nvidia/version does not exist
2023-03-10 11:59:00.718112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 142, 16)           256       
                                                                 
 conv1d_1 (Conv1D)           (None, 71, 8)             392       
                                                                 
 flatten (Flatten)           (None, 568)               0         
                                                                 
 dense (Dense)               (None, 10)                5690      
                                                                 
Total params: 6,338
Trainable params: 6,338
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (No

In [14]:
if IS_TRAINING:
    history = autoencoder.fit(train_x, train_x,
                                epochs=5, 
                                batch_size=32, 
                                verbose=1,
                                validation_split=0.1
                               )

Epoch 1/5
319/319 [==============================] - 2s 6ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0155 - mape: 587093.5625 - rmse: 0.0317 - val_loss: 7.7785e-04 - val_mse: 7.7785e-04 - val_mae: 0.0142 - val_mape: 692674.8125 - val_rmse: 0.0279
Epoch 2/5
319/319 [==============================] - 2s 6ms/step - loss: 9.3752e-04 - mse: 9.3752e-04 - mae: 0.0151 - mape: 532729.6250 - rmse: 0.0306 - val_loss: 8.2071e-04 - val_mse: 8.2071e-04 - val_mae: 0.0155 - val_mape: 636668.6250 - val_rmse: 0.0286
Epoch 3/5
319/319 [==============================] - 2s 5ms/step - loss: 8.7400e-04 - mse: 8.7400e-04 - mae: 0.0148 - mape: 484488.0625 - rmse: 0.0296 - val_loss: 7.6117e-04 - val_mse: 7.6117e-04 - val_mae: 0.0151 - val_mape: 546387.0625 - val_rmse: 0.0276
Epoch 4/5
319/319 [==============================] - 2s 5ms/step - loss: 8.1554e-04 - mse: 8.1554e-04 - mae: 0.0145 - mape: 433230.0000 - rmse: 0.0286 - val_loss: 6.6900e-04 - val_mse: 6.6900e-04 - val_mae: 0.0143 - val_mape: 509022.7188

In [15]:
if IS_TRAINING:
    from zipfile import ZipFile
    zipObj = ZipFile('sample.zip', 'w')
    autoencoder.save_weights(data_path + 'results/trained_model/%s/modelNOC' % experiment_name,"h5")

    hist_df = pd.DataFrame(history.history)
    hist_csv_file = data_path + 'results/trained_model/%s/history.csvn' % experiment_name
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
    history = history.history

In [ ]:
if not IS_TRAINING:
    history = pd.read_csv(data_path + 'results/trained_model/%s/history.csv' % RESULT_NAME)
    autoencoder.load_weights(data_path + 'results/trained_model/%s/model' % RESULT_NAME)
    autoencoder.encoder.summary()
    autoencoder.decoder.summary()

In [ ]:
if not IS_TRAINING:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(history)

### Testing

In [ ]:
results = autoencoder.evaluate(test_x, test_x, return_dict = True)
print(results)
max_rmse = 0
for index in range(test_x.shape[0]):
    result = autoencoder.evaluate(np.array([test_x[index, :, :]]), np.array([test_x[index, :, :]]), return_dict = True, verbose=0)
    max_rmse = max(max_rmse, result['rmse'])
print("Max rmse: {}".format(max_rmse))

# Results Visualization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['loss'],
                    mode='lines', name='train'))
if 'val_loss' in history:
    fig.add_trace(go.Scatter(y=history['val_loss'],
                    mode='lines', name='validation'))
fig.update_layout(title='Loss trend',
                  xaxis_title='epoch',
                  yaxis_title='loss',
                  width=1400,
                  height=600)
fig.show()

In [ ]:
train_predictions = autoencoder.predict(train_x)
labels = ['Voltage', 'Current', 'Temperature']

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[0,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_x[0,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training - battery new',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[int(train_battery_range[0]/2),:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_x[int(train_battery_range[0]/2),:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training - middle life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_predictions[train_battery_range[0]-1,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=train_x[train_battery_range[0]-1,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on training - end of life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
test_predictions = autoencoder.predict(test_x)
labels = ['Voltage', 'Current', 'Temperature']

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[0,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_x[0,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing - battery new',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[int(test_battery_range[0]/2),:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_x[0,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing - middle life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()

In [ ]:
for i in range(train_x.shape[2]):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_predictions[test_battery_range[0]-1,:,i],
                        mode='lines', name='predicted'))
    fig.add_trace(go.Scatter(y=test_x[0,:,i],
                        mode='lines', name='actual'))
    fig.update_layout(title='Results on testing - end of life',
                    xaxis_title='Step',
                    yaxis_title=labels[i],
                    width=1400,
                    height=600)
    fig.show()